# Proyecto de pruebas A/B

### Ejercicio
Has recibido una tarea analítica de una tienda en línea internacional. Tus predecesores no consiguieron completarla: lanzaron una prueba A/B y luego abandonaron (para iniciar una granja de sandías en Brasil). Solo dejaron las especificaciones técnicas y los resultados de las pruebas.

- Descripción técnica
    - Nombre de la prueba: recommender_system_test
    - Grupos: А (control), B (nuevo embudo de pago)
    - Fecha de lanzamiento: 2020-12-07
    - Fecha en la que dejaron de aceptar nuevos usuarios: 2020-12-21
    - Fecha de finalización: 2021-01-01
    - Audiencia: 15% de los nuevos usuarios de la región de la UE
    - Propósito de la prueba: probar cambios relacionados con la introducción de un sistema de recomendaciones mejorado
    - Resultado esperado: dentro de los 14 días posteriores a la inscripción, los usuarios mostrarán una mejor conversión en vistas de la página del        producto `(el evento product_page)`, instancias de agregar artículos al carrito de compras `(product_cart)` y compras `(purchase)`. En cada etapa del embudo product_page → product_cart → purchase, habrá al menos un 10% de aumento.
    - Número previsto de participantes de la prueba: 6 000

In [1]:
# Carga de librerias
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest

In [2]:
# Carga de datos
marketing_events = pd.read_csv('ab_project_marketing_events_us.csv', sep=',')
events = pd.read_csv('final_ab_events_upd_us.csv', sep=',')
new_users = pd.read_csv('final_ab_new_users_upd_us.csv', sep=',')
participants = pd.read_csv('final_ab_participants_upd_us.csv', sep=',')

# Analisis exploratorio

In [3]:

# Inspeccionar las primeras filas de cada archivo
marketing_events_head = marketing_events.head()
new_users_head = new_users.head()
events_head = events.head()
participants_head = participants.head()

marketing_events_head, new_users_head, events_head, participants_head

(                           name                   regions    start_dt  \
 0      Christmas&New Year Promo             EU, N.America  2020-12-25   
 1  St. Valentine's Day Giveaway  EU, CIS, APAC, N.America  2020-02-14   
 2        St. Patric's Day Promo             EU, N.America  2020-03-17   
 3                  Easter Promo  EU, CIS, APAC, N.America  2020-04-12   
 4             4th of July Promo                 N.America  2020-07-04   
 
     finish_dt  
 0  2021-01-03  
 1  2020-02-16  
 2  2020-03-19  
 3  2020-04-19  
 4  2020-07-11  ,
             user_id  first_date     region   device
 0  D72A72121175D8BE  2020-12-07         EU       PC
 1  F1C668619DFE6E65  2020-12-07  N.America  Android
 2  2E1BF1D4C37EA01F  2020-12-07         EU       PC
 3  50734A22C0C63768  2020-12-07         EU   iPhone
 4  E1BDDCE0DAFA2679  2020-12-07  N.America   iPhone,
             user_id             event_dt event_name  details
 0  E1BDDCE0DAFA2679  2020-12-07 20:22:03   purchase    99.99
 1  7B64

Realizar una revisión de los datos, buscando posibles problemas como tipos incorrectos, valores faltantes o duplicados.


- Verificar los tipos de las columnas.

- Revisar si hay valores faltantes.

- Identificar posibles duplicados.

In [4]:
# Verificar tipos de datos y valores faltantes en cada archivo
marketing_events_info = marketing_events.info()
print()
new_users_info = new_users.info()
print()
events_info = events.info()
print()
participants_info = participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  o

In [5]:
# Revisar si hay duplicados en cada archivo
marketing_events_duplicates = marketing_events.duplicated().sum()
new_users_duplicates = new_users.duplicated().sum()
events_duplicates = events.duplicated().sum()
participants_duplicates = participants.duplicated().sum()

Tipos de datos:
- Marketing Events: Las columnas son de tipo object, pero las fechas (start_dt, finish_dt) deberían ser de tipo datetime.

- New Users: Las columnas están bien, pero las fechas (first_date) deberían ser convertidas a tipo datetime.

- Events: Las fechas (event_dt) deben ser convertidas a datetime. Además, la columna details tiene muchos valores nulos (alrededor del 85%), lo cual puede indicar que no todos los eventos tienen información adicional.

- Participants: Todos los datos son de tipo object y están completos.

Valores faltantes:
- Marketing Events: No tiene valores faltantes.

- New Users: No tiene valores faltantes.

- Events: La columna details tiene valores faltantes, lo cual es de esperar, ya que solo los eventos de tipo purchase tendrán detalles (por ejemplo, monto de la compra).

- Participants: No tiene valores faltantes.

Duplicados:
No hay registros duplicados en ninguno de los archivos.

## Convertir las columnas de fechas a `datetime`.

In [6]:
# Convertir las columnas de fechas a tipo datetime
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])

new_users['first_date'] = pd.to_datetime(new_users['first_date'])

events['event_dt'] = pd.to_datetime(events['event_dt'])

Estudiar la distribución de eventos por usuario:

- Comprobar si el número de eventos está equilibrado entre los grupos A y B.

- Verificar si hay usuarios presentes en ambos grupos.

In [7]:
# Contar los eventos por usuario
events_per_user = events.groupby('user_id')['event_name'].count()
events_per_user.describe()

count    58703.000000
mean         7.218728
std          4.122609
min          1.000000
25%          4.000000
50%          6.000000
75%          9.000000
max         36.000000
Name: event_name, dtype: float64

El número promedio de eventos por usuario es 7.22 (con una desviación estándar de 4.12). Esto significa que la mayoría de los usuarios tienen entre 4 y 9 eventos, pero algunos usuarios tienen hasta 36 eventos.

In [8]:
# Contar los usuarios por grupo (A y B)
participants_count = participants.groupby('group')['user_id'].nunique()
participants_count

group
A    7874
B    6205
Name: user_id, dtype: int64

Grupo A (Control): 7,874 usuarios

Grupo B (Nuevo embudo de pago): 6,205 usuarios

La distribución de usuarios entre los grupos es razonablemente equilibrada, aunque el grupo A tiene un poco más de usuarios.

In [9]:
# Comprobar si hay usuarios que estén en ambos grupos
overlap_users = participants[participants.duplicated(subset='user_id', keep=False)]
overlap_users

,user_id,group,ab_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
8,B3A2485649E4A012,A,recommender_system_test
15,EAFB9027A27D510C,B,recommender_system_test
17,5D5E6EE92AF6E9E0,B,recommender_system_test
...,...,...,...
14433,7DF21AEB1AA231F9,B,interface_eu_test
14445,EA6EA431FF84563B,B,interface_eu_test
14470,70BF82527E6ED9C3,A,interface_eu_test
14487,EF2E4FAF26951358,A,interface_eu_test


Hay 1,774 usuarios que están registrados en ambos grupos A y B, lo cual es un posible problema de contaminación del grupo y podría afectar los resultados de la prueba A/B. Es importante decidir cómo manejar estos usuarios (por ejemplo, eliminarlos de uno de los grupos).

In [10]:
# Analizar la distribución de los eventos entre los días
events_per_day = events.groupby(events['event_dt'].dt.date)['event_name'].count()

events_per_day

event_dt
2020-12-07    11385
2020-12-08    12547
2020-12-09    12122
2020-12-10    14077
2020-12-11    13864
2020-12-12    17634
2020-12-13    20985
2020-12-14    26184
2020-12-15    23469
2020-12-16    20909
2020-12-17    21751
2020-12-18    22871
2020-12-19    24273
2020-12-20    26425
2020-12-21    32559
2020-12-22    29472
2020-12-23    26108
2020-12-24    19399
2020-12-26    14058
2020-12-27    12420
2020-12-28    11014
2020-12-29    10146
2020-12-30       89
Name: event_name, dtype: int64

El número de eventos aumenta progresivamente, con un máximo de 14,077 eventos registrados el 10 de diciembre de 2020.

### Excluir a los usuarios que están en ambos grupos para evitar contaminación del grupo.

In [11]:
# Excluir usuarios que están en ambos grupos (duplicados)
valid_participants = participants[~participants['user_id'].isin(overlap_users['user_id'])]

In [12]:
# Filtrar los eventos de los participantes válidos
valid_events = events[events['user_id'].isin(valid_participants['user_id'])]

In [13]:
# Contar los eventos en cada etapa del embudo
product_page_events = valid_events[valid_events['event_name'] == 'product_page']
product_cart_events = valid_events[valid_events['event_name'] == 'product_cart']
purchase_events = valid_events[valid_events['event_name'] == 'purchase']

# Conversiones (proporciones de usuarios en cada etapa)

In [14]:
# Calcular conversiones (proporciones de usuarios en cada etapa)
product_page_conversion = product_page_events['user_id'].nunique() / valid_participants['user_id'].nunique() *100
product_cart_conversion = product_cart_events['user_id'].nunique() / valid_participants['user_id'].nunique()*100
purchase_conversion = purchase_events['user_id'].nunique() / valid_participants['user_id'].nunique()*100

# Comparar las conversiones entre los grupos A y B
group_conversions = valid_participants.groupby('group').agg(
    product_page_conversion=('user_id', lambda x: product_page_events[product_page_events['user_id'].isin(x)]['user_id'].nunique() / len(x)),
    product_cart_conversion=('user_id', lambda x: product_cart_events[product_cart_events['user_id'].isin(x)]['user_id'].nunique() / len(x)),
    purchase_conversion=('user_id', lambda x: purchase_events[purchase_events['user_id'].isin(x)]['user_id'].nunique() / len(x))
)


product_page_conversion, product_cart_conversion, purchase_conversion, group_conversions

(65.69680809348286,
 32.475884244372985,
 33.605207434711005,
        product_page_conversion  product_cart_conversion  purchase_conversion
 group                                                                       
 A                     0.666291                 0.318624             0.343155
 B                     0.645281                 0.332450             0.327147)

En general, el Grupo A tiene una ligera ventaja en las tres etapas del embudo: vistas de producto, agregar al carrito y compras.

El Grupo B tiene un mejor rendimiento en la etapa de agregar al carrito (33.25% frente a 31.86% del Grupo A), pero el Grupo A tiene una mayor conversión en las etapas de vistas de producto y compras.

## Prueba estadistica

In [15]:
# Definir el número de éxitos y el número de observaciones para cada grupo y cada etapa
successes = [
    # Vistas de producto
    (group_conversions.loc['A', 'product_page_conversion'] * participants_count['A'],
     group_conversions.loc['B', 'product_page_conversion'] * participants_count['B']),
    
    # Agregar al carrito
    (group_conversions.loc['A', 'product_cart_conversion'] * participants_count['A'],
     group_conversions.loc['B', 'product_cart_conversion'] * participants_count['B']),
    
    # Compras
    (group_conversions.loc['A', 'purchase_conversion'] * participants_count['A'],
     group_conversions.loc['B', 'purchase_conversion'] * participants_count['B'])
]

In [16]:
# Definir el número total de participantes en cada grupo
n = [
    participants_count['A'],
    participants_count['B']
]


In [17]:
# Realizar la prueba Z para cada etapa
z_results = []
for success_a, success_b in successes:
    z_stat, p_value = proportions_ztest([success_a, success_b], [n[0], n[1]])
    z_results.append((z_stat, p_value))

z_results

[(np.float64(2.6072462226758417), np.float64(0.009127370064603358)),
 (np.float64(-1.7392907745918536), np.float64(0.08198362922778935)),
 (np.float64(1.996316025304993), np.float64(0.045899534801336135))]

In [20]:
# Crear una tabla con los resultados de la prueba estadística Z
z_test_results = pd.DataFrame(
    {
        'Stage': ['product_page', 'product_cart', 'purchase'],
        'Z-Statistic': [z_results[0][0], z_results[1][0], z_results[2][0]],
        'P-Value': [z_results[0][1], z_results[1][1], z_results[2][1]],
        'Significant?': ['Yes' if p < 0.05 else 'No' for _, p in z_results]
    }
)

z_test_results

,Stage,Z-Statistic,P-Value,Significant?
0,product_page,2.607246,0.009127,Yes
1,product_cart,-1.739291,0.081984,No
2,purchase,1.996316,0.045900,Yes


Resultados de la prueba Z:
- Vistas de producto (product_page):

    - Estadístico Z: 2.61

    - Valor p: 0.0091

La diferencia entre los grupos A y B en esta etapa es estadísticamente significativa (p < 0.05), lo que sugiere que el Grupo A tiene una mayor conversión en vistas de producto.

- Agregar al carrito (product_cart):

    - Estadístico Z: -1.74

    - Valor p: 0.082

La diferencia entre los grupos A y B en esta etapa no es estadísticamente significativa (p > 0.05), lo que indica que no hay suficiente evidencia para afirmar que uno de los grupos tenga una mayor conversión en esta etapa.

- Compras (purchase):

    -Estadístico Z: 2.00

    - Valor p: 0.0459

La diferencia entre los grupos A y B en esta etapa es estadísticamente significativa (p < 0.05), lo que sugiere que el Grupo A tiene una mayor conversión en compras.

- Conclusiones:
Vistas de producto: La diferencia entre los grupos es significativa, con el Grupo A mostrando una mayor conversión.

Agregar al carrito: No hay evidencia suficiente para afirmar que uno de los grupos sea superior en esta etapa.

Compras: La diferencia es significativa, y nuevamente, el Grupo A muestra una mayor conversión.

- Recomendación:
El sistema de recomendaciones en el Grupo B no parece tener un efecto positivo en las conversiones globales, ya que el Grupo A mostró mejores resultados en las etapas clave del embudo. Sin embargo, la prueba A/B no muestra una mejora sustancial en las conversiones, especialmente en la etapa de agregar al carrito.